# 📊 Analysis: Column Mapping Justification

This notebook explores the statistical and structural relationships between the columns to justify the inferred mapping.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv("data.csv")
df.describe()


### 🔍 Step 1: Identifying Volume

In [ ]:
volume_counts = {}
columns = df.columns
for col in columns:
    volume_counts[col]=0

for index,row in df.iterrows():
    candidates = []
    for col in columns:
        value = row[col]
        if value > 0 and value == int(value):
            candidates.append(col)
    if candidates:
        max_value = 0
        best_col = None
        for col in candidates:
            if row[col] > max_value:
                max_value = row[col]
                best_col = col
        volume_counts[best_col] += 1
                    
for col,count in volume_counts.items():
    volume_counts[col] = count / len(df)
max_column_volume = max(volume_counts, key=volume_counts.get)
max_confidence = volume_counts[max_column_volume]

plt.figure(figsize=(10, 6))
plt.bar(volume_counts.keys(), volume_counts.values())
plt.title('Probability of Each Column Being Volume')
plt.xlabel('Columns')
plt.ylabel('Probability')
plt.tight_layout()
plt.show()

print(f"Thus, Column {max_column_volume} is the most likely candidate for volume with confidence {max_confidence}")

### 🔍 Step 2: Correlation Analysis

In [ ]:
df.columns = df.columns.str.strip()

print("\n1. Pearson Correlation Matrix:")
pearson_corr = df.corr(method='pearson')
print(pearson_corr.round(5))

print("\n2. Spearman Correlation Matrix:")
spearman_corr = df.corr(method='spearman')
print(spearman_corr.round(5))

plt.figure(figsize=(15, 13))

plt.subplot(2, 2, 1)
sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', center=0, square=True, fmt='.3f', cbar_kws={'label': 'Correlation'})
plt.title('Pearson Correlation Matrix')

plt.subplot(2, 2, 2)
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', center=0, square=True, fmt='.3f', cbar_kws={'label': 'Correlation'})
plt.title('Spearman Correlation Matrix')

### 🔍 Step 3: Row-wise Statistics inference (as applicable)

In [ ]:
filtered_columns = [col for col in columns if col != max_column_volume]
incomings = {col : 0 for col in filtered_columns}
outgoings = {col : 0 for col in filtered_columns}

for high_col in filtered_columns:
    for low_col in filtered_columns:
        if high_col == low_col:
            continue
            
        if (df[high_col] >= df[low_col]).all():
            print(f"{high_col} >= {low_col} for all rows")
            incomings[low_col] += 1
            outgoings[high_col] += 1

filtered_low = [col for col, value in incomings.items() if value == max(incomings.values())]
filtered_high = [col for col, value in outgoings.items() if value == max(outgoings.values())]

print(f"\nMost incoming column: {filtered_low} with {max(incomings.values())} incoming connections")
print(f"Most outgoing column: {filtered_high} with {max(outgoings.values()) } outgoing connections")

In [ ]:
columns = df.columns.tolist()
print("Columns:", columns)

filtered_columns = [col for col in columns if col != max_column_volume]
larger = {}
smaller = {}
vol_pulse_large = 0
vol_pulse_small = 0
double_max = 0
double_min = 0
for col in filtered_columns:
    larger[col] = 0
    smaller[col] = 0
for index,row in df.iterrows():
    max_value = -212
    min_value = 290
    max_col = None
    min_col = None
    for col in filtered_columns:
        if max_value < row[col]:
            max_value = row[col]
            max_col = col
        if min_value > row[col]:
            min_value = row[col]
            min_col = col
    larger[max_col] += 1
    smaller[min_col] += 1
    if max_col == "pulse":
        vol_pulse_large += row[max_column_volume]
    if min_col == "pulse":
        vol_pulse_small += row[max_column_volume]

for col in filtered_columns:
    print(f"Column {col} is largest {larger[col]} times and smallest {smaller[col]} times")

print(f"Avg volume when pulse is high: {vol_pulse_large / larger['pulse']:.2f}")
print(f"Avg volume when pulse is low: {vol_pulse_small / smaller['pulse']:.2f}")

for col in filtered_columns:
    larger[col] = larger[col] / len(df)
    smaller[col] = smaller[col] / len(df)

large_price = 0
max_col_high = None
max_col_low = None
small_price = 0
for col in filtered_columns:
    if large_price < larger[col]:
        large_price = larger[col]
        max_col_high = col
    if small_price < smaller[col]:
        small_price = smaller[col]
        max_col_low = col

max_col_price = None
for col in filtered_columns:
    if larger[col] > 0 or smaller[col] > 0:
        max_col_price = col
        break
        
plt.figure(figsize=(10, 6))
plt.bar(larger.keys(), larger.values())
plt.title('Probability of Each Column Being High')
plt.xlabel('Columns')
plt.ylabel('Probability')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plt.bar(smaller.keys(), smaller.values())
plt.title('Probability of Each Column Being Low')
plt.xlabel('Columns')
plt.ylabel('Probability')
plt.tight_layout()
plt.show()

In [ ]:
candidate_open_close = [col for col in filtered_columns if col != max_col_high and col != max_col_low and col != max_col_price]
candidate_open = candidate_open_close[0]
candidate_close = candidate_open_close[1]
same1 = 0
for i in range(len(df)-1):
    o1 = df.iloc[i][candidate_open]
    o2 = df.iloc[i+1][candidate_open]
    c1 = df.iloc[i][candidate_close]
    c2 = df.iloc[i+1][candidate_close]
    if o1  > c1 and o2 > c2 and o1 > c2:
        same1 += 1
    elif o1 < c1 and o2 < c2 and o1 < c2:
        same1 += 1

candidate_open = candidate_open_close[1]
candidate_close = candidate_open_close[0]
same2 = 0
for i in range(len(df)-1):
    o1 = df.iloc[i][candidate_open]
    o2 = df.iloc[i+1][candidate_open]
    c1 = df.iloc[i][candidate_close]
    c2 = df.iloc[i+1][candidate_close]
    if o1  > c1 and o2 > c2 and o1 > c2:
        same2 += 1
    elif o1 < c1 and o2 < c2 and o1 < c2:
        same2 += 1

print(f"")
print(f"Same1 absolute: {same1}, Same2 absolute: {same2}")
print(f"Same1: {same1/(len(df)-1)}, Same2: {same2/(len(df)-1)}")


In [ ]:
size = len(df)

candidate_open = candidate_open_close[0]
candidate_close = candidate_open_close[1]
works1 = 0
for i in range(0,size,1000):
    row1 = df.iloc[i]
    row2 = df.iloc[i+999]
    total_diff = row2[candidate_close] - row1[candidate_open]
    candidate_sum = 0
    for j in range(i,i+1000):
        candidate_sum += df.iloc[j][candidate_close] - df.iloc[j][candidate_open]
    if(candidate_sum > 0 and total_diff > 0):
        works1+= 1
    elif (candidate_sum < 0 and total_diff < 0):
        works1+= 1

candidate_open = candidate_open_close[1]
candidate_close = candidate_open_close[0]
works2 = 0
for i in range(0,size,1000):
    row1 = df.iloc[i]
    row2 = df.iloc[i+999]
    total_diff = row2[candidate_close] - row1[candidate_open]
    candidate_sum = 0
    for j in range(i,i+1000):
        candidate_sum += df.iloc[j][candidate_close] - df.iloc[j][candidate_open]
    if(candidate_sum > 0 and total_diff > 0):
        works2+= 1 
    elif (candidate_sum < 0 and total_diff < 0):
        works2+= 1

print(f"Works1 absolute: {works1}, Works2 absolute: {works2}")



In [ ]:
df.columns = df.columns.str.strip()
columns = df.columns.tolist()

print("Columns:", columns)

df_prev = df.shift(1)

results = []

for open_col in ['deltaX', 'flux', 'pulse']:
    for close_col in ['deltaX', 'flux', 'pulse']:
        if open_col == close_col:        
            continue

        price_col = ['deltaX', 'flux', 'pulse'].copy()
        price_col.remove(open_col)
        price_col.remove(close_col)
        price_col = price_col[0]

        closecount = 0
        opencount = 0
        
        open_distance = abs(df[price_col] - df_prev[open_col])
        close_distance = abs(df[price_col] - df_prev[close_col])
            
        closer_to_open = (open_distance < close_distance).sum()
        closer_to_close = (close_distance < open_distance).sum()
            
        opencount += closer_to_open
        closecount += closer_to_close
        
        print(f"\nOpen: {open_col}, Close: {close_col}, Price: {price_col}")
        print(f"Closer to Open: {opencount}")
        print(f"Closer to Close: {closecount}")
        print(f"Open % (lower the better): {100* opencount / (opencount + closecount):.2f}")

### 📈 Step 4: Candlestick Visual from Mapped Fields

In [ ]:
# Add your logic here
# . . . .
# sample[['open', 'high', 'low', 'close']].plot(figsize=(12,6), title='Candlestick Field Preview')
# plt.show()

This confirms the structural pattern of price movement and supports the inferred mapping.